# 2. Word2Vec в качестве признаков

In [1]:
import os,sys
import pandas as pd
import numpy as np
from collections import Counter
import nltk
import string
import random
from nltk.corpus import stopwords
import nltk.tokenize as word_tokenize
from tqdm import tqdm

from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("russian") # Choose a language

from matplotlib import pyplot as plt
%matplotlib inline

from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec

import os.path


Using TensorFlow backend.


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [3]:
import re 
import emoji

In [4]:
from matplotlib import pyplot as plt
%matplotlib inline

Скользящий контроль

In [5]:
from get_fold import get_fold #разбивка на выборку с нормальным распределением
def getTrainTest(seed = 42):  #создать тестовую и тренировучную выборку
    X_train = pd.concat([get_fold(K_fold=i,seed = seed) for i in range(7)])
    X_test  = pd.concat([get_fold(K_fold=i,seed = seed) for i in range(7,10)])
    return X_train, X_test

## Word2Vec, обученный на википедии

In [3]:
import gensim

w2v_fpath = "w2v/all.norm-sz100-w10-cb0-it1-min100.w2v"
w2v = gensim.models.KeyedVectors.load_word2vec_format(w2v_fpath, binary=True, unicode_errors='ignore')
w2v.init_sims(replace=True)

## Word2Vec, обученный на постах вконтакте

In [11]:
def decode(a, encoding="utf8"):
    if isinstance(a, bytes):
        return a.decode(encoding)
    else:
        return a

    return [decode(i) for i in tokens]


def tokenize_w2v(file_text):   #Токенизация предложений
    tokens = nltk.word_tokenize(file_text, language='english')
    tokens = [i for i in tokens if ( i not in string.punctuation )]

    #cleaning words
    tokens = [i.replace(u"«", u"").replace(u"»", u"") for i in tokens]
    tokens_new = []
    for i in tokens:
#         if i in w2v.wv:
        smiles = re.findall(emoji.get_emoji_regexp(), i)
        while '' in smiles:
            smiles.remove('')
        if len(smiles) == 0:
            tokens_new.append(i)
        else:
            temp = i
            for s in smiles:
                temp = temp.replace(s,'')
            if temp != '':
                tokens_new.extend([temp]+smiles)
            else:
                tokens_new.extend(smiles)
    return tokens_new

## Обучение на постах вконтакте

In [13]:
# # Обучение на постах вконтакте
import json
posts = json.load(open('data/posts_list.txt','r'))
print('loading posts')
sentences = []
for i in tqdm(posts):
    sentences.append(tokenize_w2v(decode(i).lower()))
print('making sentences')
import gensim
w2v = gensim.models.Word2Vec(sentences, size=100, window=5, min_count=3, workers=4)
print('model')
w2v.wv.save_word2vec_format('w2v/model_w2v_vk.mod')

model


## Загрузка предобученной модели вконтакте

In [ ]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('w2v/model_w2v_vk.mod')

## Логгирование

In [14]:
metrics = ['accuracy_score', 'f1_score', 'precision_score', 'recall_score']
types =   ['0/1/-1', '0/1,-1','1/0,-1','-1/0,1']

def make_log(y_predict, y_test, seed = 42):
    text =  "\n\nseed\t"+str(seed)
    text += "\n'3 class classification\t0/1/-1"
    text += "\naccuracy_score\t"+str(accuracy_score(y_test, y_predict))
    text += "\nf1_score\t"+str(f1_score(y_test, y_predict, average = 'micro'))
    text += "\nprecision_score\t"+str(precision_score(y_test, y_predict, average = 'micro'))
    text += "\nrecall_score\t"+str(recall_score(y_test, y_predict, average = 'micro'))
    
    text += "\n2 class classification\t0/1,-1"
    lbl = 0
    text += "\naccuracy_score\t"+str(accuracy_score([i==lbl for i in  y_test], [i==lbl for i in  y_predict]))
    text += "\nf1_score\t"+str(f1_score([i==lbl for i in  y_test], [i==lbl for i in  y_predict]))
    text += "\nprecision_score\t"+str(precision_score([i==lbl for i in  y_test], [i==lbl for i in  y_predict]))
    text += "\nrecall_score\t"+str(recall_score([i==lbl for i in  y_test], [i==lbl for i in  y_predict]))

    
    text += "\n2 class classification\t1/0,-1"
    lbl = 1
    text += "\naccuracy_score\t"+str(accuracy_score([i==lbl for i in  y_test], [i==lbl for i in  y_predict]))
    text += "\nf1_score\t"+str(f1_score([i==lbl for i in  y_test], [i==lbl for i in  y_predict]))
    text += "\nprecision_score\t"+str(precision_score([i==lbl for i in  y_test], [i==lbl for i in  y_predict]))
    text += "\nrecall_score\t"+str(recall_score([i==lbl for i in  y_test], [i==lbl for i in  y_predict]))

    text += "\n2 class classification\t-1/0,1"
    lbl = -1
    text += "\naccuracy_score\t"+str(accuracy_score([i==lbl for i in  y_test], [i==lbl for i in  y_predict]))
    text += "\nf1_score\t"+str(f1_score([i==lbl for i in  y_test], [i==lbl for i in  y_predict]))
    text += "\nprecision_score\t"+str(precision_score([i==lbl for i in  y_test], [i==lbl for i in  y_predict]))
    text += "\nrecall_score\t"+str(recall_score([i==lbl for i in  y_test], [i==lbl for i in  y_predict]))
   
    text += "\nmatrix_cross_valid\n"
    M = np.zeros((3,3), dtype = int)
    for i in zip(y_test, y_predict):
        t1, t2 = i
        if i[0] == -1:
            t1 = 2
        if i[1] == -1:
            t2 = 2
        M[t1, t2] += 1
    text += str(M[0,0])+','+str(M[0,1])+','+str(M[0,2])+'\n'
    text += str(M[1,0])+','+str(M[1,1])+','+str(M[1,2])+'\n'
    text += str(M[2,0])+','+str(M[2,1])+','+str(M[2,2])+'\n'

    return text

## Создание выборки для обучения 

In [17]:
def decode(a, encoding="utf8"):
    if isinstance(a, bytes):
        return a.decode(encoding)
    else:
        return a

    return [decode(i) for i in tokens]

def tokenize_me(file_text):
    tokens = nltk.word_tokenize(file_text, language='english')
    tokens = [i.replace(u"«", u"").replace(u"»", u"") for i in tokens]
    tokens_new = []
    for i in tokens:
        smiles = re.findall(emoji.get_emoji_regexp(), i)
        while '' in smiles:
            smiles.remove('')
        if len(smiles) == 0:
            tokens_new.append(i)
        else:
            temp = i
            for s in smiles:
                temp = temp.replace(s,'')
            if temp != '':
                tokens_new.extend([temp]+smiles)
            else:
                tokens_new.extend(smiles)
    tokens = []
    for w in tokens_new:
        if w in w2v.wv:
            tokens.append(w)
    return tokens

def get_words_matrix(df, show_progress = True):
    all_words = []
    if show_progress:
        for i in tqdm(df['data'].get_values(), leave = False):
            words = []
            words = tokenize_me(decode(i).lower())
            all_words.extend(words)   
    else:
        for i in df['data'].get_values():
            words = []
            words = tokenize_me(decode(i).lower())
            all_words.extend(words)   
    all_words = Counter(all_words)
    all_words = dict(filter(lambda x:x[1]>1 and x[1]<250, all_words.items()))
    print("Len: "+str(len(all_words)))
    return list(all_words.keys())

def get_X_matrix(df, all_words, show_progress = False, dtype = 1):
    X_train = np.zeros((len(df), w2v.wv.word_vec('прив').shape[0]))
    if show_progress:
        for c,i in tqdm(enumerate(df['data'].get_values()), leave = False):
            words = []
            words = tokenize_me(decode(i).lower())
            for w in words:
                if w in all_words:
                    ind = all_words.index(w)
                    X_train[c] += w2v.wv.word_vec(w)
    else:
        for c,i in enumerate(df['data'].get_values()):
            words = []
            words = tokenize_me(decode(i).lower())
            for w in words:
                if w in all_words:
                    ind = all_words.index(w)
                    X_train[c] += w2v.wv.word_vec(w)
    return X_train      


## Нейронная сеть в качестве алгоритма обучения

In [18]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD


def get_label(lst):
    ind = lst.argmax()
    if ind == 0:
        return 0
    elif ind == 1:
        return 1
    elif ind == 2:
        return -1
    
def NN(X_train, y_train, X_test, y_test, verbose = 0, epochs = 50):
	N_out = 3
	N_in  = X_train.shape[1]
	X_all = np.concatenate((X_train,X_test))
	y_all = keras.utils.to_categorical(np.concatenate((y_train,y_test)), num_classes=N_out)
	model = Sequential()
	model.add(Dense(512, activation='relu', input_dim=N_in))
# 	model.add(Dropout(0.5))
	model.add(Dense(128, activation='relu'))
# 	model.add(Dropout(0.5))
	model.add(Dense(N_out, activation='softmax'))
	sgd = SGD(lr=0.01, decay=1e-6, momentum=0.8, nesterov=True)
	model.compile(loss='categorical_crossentropy',
	              optimizer=sgd,
	              metrics=['accuracy'])

# 	#### Draw model graph
# 	from keras.utils.vis_utils import plot_model  
# 	plot_model(model, to_file='model.png', show_shapes=True)  
# 	#####################


	history = model.fit(X_all,
	 	 		 	 	y_all, 
	 	 	 	 	 	validation_split = 0.3,
	 	 	 	 	 	epochs=epochs,
	 	 	 	 	 	batch_size=64,
	 	 	 	 	 	verbose=verbose,
	 	 	 	 	 	shuffle=False)
	print('===')  
	y_predict = [get_label(k) for k in model.predict(X_test, batch_size=32, verbose=0)]
	return y_predict  

## Интерфейс для логистической регрессии, SVM и нейронной сети

In [19]:
def LogisticRegressionScore(X_train, y_train, X_test, y_test):
    regr = LogisticRegression(C = 0.4)
    regr.fit(X_train, y_train)
    y_predict = regr.predict(X_test)
    temp = make_log(y_predict, y_test)
    return temp
def SVMScore(X_train, y_train, X_test, y_test):
    SVM_model = LinearSVC(C = 0.05)
    SVM_model.fit(X_train, y_train)
    y_predict = SVM_model.predict(X_test)
    temp = make_log(y_predict, y_test)
    return temp

def NNScore(X_train, y_train, X_test, y_test, verbose = 1, epochs = 10):
    y_predict = NN(X_train, y_train, X_test, y_test, verbose = verbose, epochs=epochs)
    temp = make_log(y_predict, y_test)
    return temp

def LR_SVM_NN_logs(X_train, y_train, X_test, y_test):
    return (LogisticRegressionScore(X_train, y_train, X_test, y_test), 
            SVMScore(X_train, y_train, X_test, y_test), 
            NNScore(X_train, y_train, X_test, y_test)
            )

## Создание признаков, обучение алгоритмов и логирование результатов в файлы 

In [20]:
SVM_log    = ""
LOGREG_log = ""
NN_log     = ""
for e in range(100):
    print("Iteration:\t"+str(e))
    df_train, df_test  = getTrainTest(seed = e)
    dtype = 2
    all_words = get_words_matrix(df_train, show_progress = False)
    print('Train')
    X_train = get_X_matrix(df_train, all_words, show_progress = False)
    print('Test')
    X_test  = get_X_matrix(df_test, all_words, show_progress = False)
    y_train = df_train['label']
    y_test  = df_test['label']
    
    print('LogReg')
    LOGREG_log += LogisticRegressionScore(X_train, y_train, X_test, y_test)
    
    print("SVM")
    SVM_log += SVMScore(X_train, y_train, X_test, y_test)
    
    print("Neural network")
    NN_log += NNScore(X_train, y_train, X_test, y_test, verbose=0, epochs=13)


    open('logs/LogReglogsW2V_vkmy.txt','w+').write(LOGREG_log)
    open('logs/SVMlogsW2V_vkmy.txt','w+').write(SVM_log)
    open('logs/NNlogsW2V_vkmy.txt','w+').write(NN_log)

Iteration:	0
Len: 9104
Train
Test
LogReg
SVM
===
Iteration:	1
Len: 9275
Train
Test
LogReg
SVM
===
Iteration:	2
Len: 9116
Train
Test
LogReg
SVM
===
Iteration:	3
Len: 9466
Train
Test
LogReg
SVM
===
Iteration:	4
Len: 9610
Train
Test
LogReg
SVM
===
Iteration:	5
Len: 9028
Train
Test
LogReg
SVM
===
Iteration:	6
Len: 9773
Train
Test
LogReg
SVM
===
Iteration:	7
Len: 9646
Train
Test
LogReg
SVM
===
Iteration:	8
Len: 9681
Train
Test
LogReg
SVM
===
Iteration:	9
Len: 9691
Train
Test
LogReg
SVM
===
Iteration:	10
Len: 9510
Train
Test
LogReg
SVM
===
Iteration:	11
Len: 9194
Train
Test
LogReg
SVM
===
Iteration:	12
Len: 9200
Train
Test
LogReg
SVM
===
Iteration:	13
Len: 9818
Train
Test
LogReg
SVM
===
Iteration:	14
Len: 9334
Train
Test
LogReg
SVM
===
Iteration:	15
Len: 9190
Train
Test
LogReg
SVM
===
Iteration:	16
Len: 9641
Train
Test
LogReg
SVM
===
Iteration:	17
Len: 9441
Train
Test
LogReg
SVM
===
Iteration:	18
Len: 9653
Train
Test
LogReg
SVM
===
Iteration:	19
Len: 9166
Train
Test
LogReg
SVM
===
Iteration: